In [32]:
!pip install torch_geometric

import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),    # normal users community
         (3,4), (4,5), (5, 3)]    # suspicious cluster
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (e.g., train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

In [33]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

GCN Test Accuracy: 0.68



**Improvements**

Increased hidden layer size
Changed the first GCN layer’s output from 4 to 16 to give the model more learning capacity.

Added dropout(Learning rate) (0.5)
Helps prevent overfitting and improves generalization on unseen nodes.

Added weight decay (L2 regularization)
Using weight_decay=5e-4 stabilizes training and improves accuracy.

Increased training epochs from 50 to 200
Allows the GCN to fully converge and learn better node representations.

Improved training structure
Ensures more stable training and consistent accuracy improvements.

In [34]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class ImprovedGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)   # bigger hidden layer
        self.conv2 = GCNConv(16, dataset.num_classes)
        self.dropout = 0.5

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = ImprovedGCN()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop
for epoch in range(200):  # more epochs for stability
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')


GCN Test Accuracy: 0.81
